In [42]:
import PyPDF2 as p2
from pdfminer.pdfparser import PDFParser
from pdfminer.pdfdocument import PDFDocument
import pandas as pd
import os
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# from goto import goto, label

In [43]:
#define the paths of the PDF scrapped 

path = 'F:/Environmental Baseline Data/Version 3/Data/PDF'
pdfs = os.listdir(path)
pdf_paths = [str(path) + '/' + str(x) for x in pdfs]
len(pdf_paths)

1329

# 1. Make Outlines Based on the Postal Codes 

This part of the code will generate a csv file with all the PDF nammes with the fillowing details: <br> - 'file_nos': just a number to indicate the file
<br> - 'file_names': the name of the file (Postal Code.pdf)
<br> - 'file_paths' : the path of the pdf file
<br> - 'int_postals' : the integer value of the postal code
<br> - 'chunk_nos': chunk_no based on the consectutive Postal Code
    

### Postal to Int Functions 

In [44]:
def postal_to_int(Postal):
    multiplier = 1
    odd = 1
    Postal_Int_Word = 0
    Error_Message = "Invalid Postal Code"
    
    # Reversing the string 
    Postal = Postal[::-1] 
    
    # Chcek for the length of the Postal Code
    if not(len(Postal)==6):
        return(Error_Message)
    
    for letter in list(str.strip(Postal)):
        if odd%2 == 1:
            # Check if it is an interger or not   
            # If this is not an integer then exit
            if not (str.isnumeric(letter)):
                return(Error_Message)
            
            Postal_Int_Letter = multiplier * int(letter)
            Postal_Int_Word = Postal_Int_Word + Postal_Int_Letter
            
            odd = odd + 1
            multiplier = multiplier * 10
            # print(letter, Postal_Int_Letter)
            # print(Postal_Int_Word)
        else:
            # Check if it is a character or not
            # If this is not a character then exit
            if not (str.isalpha(letter)):
                return(Error_Message)
            
            # ord('A') = 65
            Postal_Int_Letter = multiplier * (ord(letter) - 65)
            Postal_Int_Word = Postal_Int_Word + Postal_Int_Letter
            
            odd = odd + 1
            multiplier = multiplier * 26
            # print(letter, Postal_Int_Letter)
            # print(Postal_Int_Word)
            
    return Postal_Int_Word

In [39]:
# never used integer to Postal but I guess it would be useful 
# Also useful in validating if Postal to Int Function works 

def int_to_postal(Postal_Int_Word):
    Postal_Int_Word_Rem = Postal_Int_Word
    Postal = ""
    odd = 0
    Error_Message = "Invalid Postal Code"
    
    # Reversing the string 
    Postal = Postal[::-1] 
 
    while odd < 6:
        if odd%2 == 0:
            Postal_Int_Letter = Postal_Int_Word_Rem % 10
            Postal = Postal + str(Postal_Int_Letter)
            Postal_Int_Word_Rem = int(Postal_Int_Word_Rem/10)
            odd = odd + 1

        else:
            Postal_Int_Letter = (Postal_Int_Word_Rem % 26) +65
            Postal = Postal + chr(Postal_Int_Letter)
            Postal_Int_Word_Rem = int(Postal_Int_Word_Rem/26)
            odd = odd + 1
            
    # Accounts for 7 digit Postal Codes
    if Postal_Int_Word_Rem > 0:
        return(Error_Message)
        
    return Postal[::-1]
  

In [40]:
file_nos = []
file_names = []
file_paths = []
int_postals = []
file_no = 1

for pdf_path in pdf_paths:
    # additional check to get rid of the string "Thumb"
    if 'Thum' in pdf_path:
        continue
    
    file_nos.append(file_no)
    file_name = str(pdf_path.split('/')[-1])
    file_names.append(file_name)
    file_paths.append(pdf_path)
    
    postal = file_name[:-4]
    int_postal = postal_to_int(postal)
    int_postals.append(int_postal)
    
len(file_nos)
len(file_names)
len(file_paths)
len(int_postals)

1328

1328

1328

1328

In [41]:
chunk_nos = []
chunk = 0
last_postal = int_postals[0]

for int_postal in int_postals:
    new_chunk_found = 0
    if int_postal == last_postal + 1:
        chunk_nos.append(chunk)
    else:
        new_chunk_found = 1
        chunk = chunk +1
        chunk_nos.append(chunk)
    if new_chunk_found == 0 and 
    last_postal = int_postal
    
len(chunk_nos)
chunk

1328

86

In [35]:
df_outlines = pd.DataFrame({'file_nos': file_nos, 'file_names': file_names, 'file_paths' : file_paths, 'int_postals': int_postals, 'chunk_nos': chunk_nos})
df_outlines.head(5)
df_outlines.to_csv("F:/Environmental Baseline Data/Version 3/Indices/Outline_Present.csv")

,file_nos,file_names,file_paths,int_postals,chunk_nos
0,1,A0H8C0.pdf,F:/Environmental Baseline Data/Version 3/Data/...,20300,1
1,1,A0H8C1.pdf,F:/Environmental Baseline Data/Version 3/Data/...,20301,1
2,1,A0H8C2.pdf,F:/Environmental Baseline Data/Version 3/Data/...,20302,1
3,1,A0H8C3.pdf,F:/Environmental Baseline Data/Version 3/Data/...,20303,1
4,1,A0U3G1.pdf,F:/Environmental Baseline Data/Version 3/Data/...,52841,2


# 2. Extract Chapters (Outline level < 4) from all PDFs 

In [15]:
# Functtion that runs recursively to extract the outlines of the PDF files
def nested_is_instance_check(s, level, file_name, file_names_ch, chapters, levels, pages):
    level +=1
    if isinstance(s, (int, list, float, complex)):
        for x in s:
            nested_is_instance_check(x, level, file_name, file_names_ch, chapters, levels, pages)
    else:
        chapters.extend([s.title])
        levels.extend([level])
        #it was observed that for a lot of PDF files we could not find the page number 
        # Hence, -999 is the error code added for those cases 
        try:
            pages.extend([(pdfread.getDestinationPageNumber(s))])
        except: 
            pages.extend([-999])
        file_names_ch.append([file_name])

In [16]:
def count_occorances(lst, x):
    count = 0
    for ele in lst:
        if ele == x:
            count = count + 1
    return count

In [17]:
current_df_outlines = df_outlines[df_outlines.chunk_nos == 3]
type(current_df_outlines.file_paths)

pandas.core.series.Series

In [25]:
outline_present = []
outline_not_present_but_TOC_present = []
file_names = []

# Variables for Initialization of the data frame for Index 4 - Chapters.csv
file_names_ch = []
chapters = []
levels = []
pages = []
all_df_chapters = pd.DataFrame({'file_names': file_names_ch, 'chapters': chapters, 'pages' : pages, 'levels' : levels})


for current_chunk in range(0, chunk+1):
    current_df_outlines = df_outlines[df_outlines.chunk_nos == current_chunk]
    
    # Variables for Index 4 - Chapters.csv for each Chunk
    file_names_ch = []
    chapters = []
    levels = []
    pages = []
    
    
    last_file_names_ch = []
    last_chapters = []
    last_levels = []
    last_pages = []
    
    # Going Chunk by Chunk 
    # We define chunk as all pdfs that have consecutive Postal Codes 
    for pdf_path in current_df_outlines.file_paths:
        
        len_s = -99
        
        # Variables for Index 4 - Chapters.csv for each Chunk
        p_file_names_ch = []
        p_chapters = []
        p_levels = []
        p_pages = []
               
        #Initializations are done here 
        file_name = str(pdf_path.split('/')[-1])
        file_names.append(file_name)
        outline_p = 0
        
        ### Try getting the outline first 
        try:
            pdfread = p2.PdfFileReader(pdf_path)
            s = pdfread.outlines
            len_s = len(s)
        except:
            len_s = -99
        
        if len_s > 0:
            level = 0
            outline_p = 1
            outline_present.append(1) # 1 Indicating that the outline is present 
            outline_not_present_but_TOC_present.append(0)
            nested_is_instance_check(s, level, file_name, p_file_names_ch, p_chapters, p_levels, p_pages)
        elif len_s == 0:
            outline_p = 0
            outline_present.append(0)
        else:
            outline_p = -1
            outline_present.append(-1)
    
        #######################################################################
        ##### These are cases in which the outline for that PDF is present ####
        #######################################################################
        
        ## If Outline is present check if the last pdf had the same Outline
        if outline_p > 0:
            if count_occorances(p_pages, -999) > 0:
            ###################################################
            ############ Write a fn  to look for the same stuff in other pdf files. 
            ############
        
                # look if other PDFs have outlines too
                if p_chapters == last_chapters:
                    print("Outline Repeatition confirmed between {} and {}".format(last_file_names_ch[0], p_file_names_ch[0]))
                    ### Retain the chapter names from last chapter and keep moving 
                    chapters_might_modify = 1
                    for i in range(len(last_pages)):
                        if last_pages[i] < 0 and p_pages[i] < 0:
                            last_pages[i] = '{} - {}'.format(p_file_names_ch, str(p_pages[i]))
                    
                ###################################################
                ############ Write a fn  to make it (page_nos) consistent
                ############
                else:
                    print("This file {} might carry on to other PDFs".format(file_name))
                    last_chapters = p_chapters
                    last_file_names_ch = p_file_names_ch
                    last_levels = p_levels
                    last_pages = p_pages
                    
            else:
                file_names_ch.extend(p_file_names_ch)
                chapters.extend(p_chapters)
                levels.extend(p_levels)
                pages.extend(p_pages)
                last_chapters = []
                continue
                chapters_might_modify = 0
                
                
            if count_occorances(last_pages, -999) < 1:
                file_names_ch.extend(last_file_names_ch)
                chapters.extend(last_chapters)
                levels.extend(last_levels)
                pages.extend(last_pages)
                last_chapters = []
                continue
                chapters_might_modify = 0
                
            last_chapters = p_chapters

        
        ###########################################################################
        ##### These are cases in which the outline for that PDF is not present ####
        ###########################################################################
    
        ## If outline is not present then let's see if theer are other ways to check             
        else:
            ###################################################
            ############ Use Other methods to extract the TOC 
            ### 1. Check in the chunk for help 
            match_in_last_outline = 0
            
            if match_in_last_outline == 0:
                file_names_ch.extend(last_file_names_ch)
                chapters.extend(last_chapters)
                levels.extend(last_levels)
                pages.extend(last_pages)
                last_chapters = []
                continue
                chapters_might_modify = 0
                
            ### 2. Check for PDF to HTMl Method to extract Table of contents 
            # if TOC Present then outline_not_present_but_TOC_present.append(1)
            outline_not_present_but_TOC_present.append(1)
                
                
                
        
        

            
            
                
        # Error Code for s (=pdfraed.outlines) is -99
        # the negative number allows exclusivity from the if and the elif conditions above
        # so the previous value is not affecting the next itteration in case the try block 
        # does not complete all its operations 
        
             
            
    # change this based on the above mentioned if statements 
    df_chapters = pd.DataFrame({'file_names': file_names_ch, 'chapters': chapters, 'pages' : pages, 'levels' : levels})
    all_df_chapters = pd.concat([all_df_chapters, df_chapters])
    
        
        


This file A0H8C0.pdf might carry on to other PDFs
Outline Repeatition confirmed between ['A0H8C0.pdf'] and ['A0H8C1.pdf']
This file A0Z0Z2.pdf might carry on to other PDFs
This file A1C3X9.pdf might carry on to other PDFs
This file A1C3Y1.pdf might carry on to other PDFs
This file A1J6I6.pdf might carry on to other PDFs
This file A1T0F2.pdf might carry on to other PDFs
This file A1T0F5.pdf might carry on to other PDFs
This file A1T0F7.pdf might carry on to other PDFs
This file A1X1X2.pdf might carry on to other PDFs
This file A1X1Y1.pdf might carry on to other PDFs
This file A1X1Y2.pdf might carry on to other PDFs
This file A1Y9H5.pdf might carry on to other PDFs
This file A2F4L2.pdf might carry on to other PDFs
This file A3E2X6.pdf might carry on to other PDFs
This file A3S2I9.pdf might carry on to other PDFs
This file A3S2J5.pdf might carry on to other PDFs
This file A3S2R5.pdf might carry on to other PDFs
This file A3S2R6.pdf might carry on to other PDFs
This file A4E6W9.pdf might c

In [26]:
len(outline_present)
len(file_names)
len(df_outlines['file_names'])

1328

1328

1328

In [27]:
df_outlines['outline_present'] = outline_present

In [28]:
##########################################################################################
########## We might need to think of a better way to extract important chapters ##########
##########  df.chapters.levels < 5, is not teh best way ##################################
##########################################################################################
all_df_chapters = all_df_chapters[all_df_chapters.chapters is not None  and all_df_chapters.levels < 5]
all_df_chapters.head(5)
all_df_chapters.to_csv("F:/Environmental Baseline Data/Version 3/Indices/Index 4 (Outlines)- Chapters.csv")

  
 

,file_names,chapters,pages,levels
0,[A0Y1T7.pdf],EXECUTIVE SUMMARY,1.0,2.0
1,[A0Y1T7.pdf],1.0 INTRODUCTION,21.0,2.0
2,[A0Y1T7.pdf],1.1 Overview,21.0,3.0
3,[A0Y1T7.pdf],1.2 Project Need,21.0,3.0
4,[A0Y1T7.pdf],1.3 Regulatory Framework,21.0,3.0
